In [1]:
# Importing modules
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
from pyproj import CRS
import contextily as ctx


In [2]:
# Reading the data into a dataframe
data = pd.read_json('https://iot.fvh.fi/opendata/uiras/uiras-meta.json', 
                    orient="index") 

# Creating shapely Point objects from the coordinates
data['geometry'] = data.apply(lambda row: 
                              Point(row['lon'], row['lat']), axis=1)

# Creating a geodataframe
geo = gpd.GeoDataFrame(data, 
                       geometry='geometry')

# Setting the CRS to EPSG:3857
geo = geo.set_crs(epsg=4326)
geo = geo.to_crs(epsg=3857)


In [3]:
geo

,name,lat,lon,servicemap_url,site_url,fieldmap,geometry
70B3D57050001AB9,Pikkukosken uimaranta,60.227704,24.983821,https://palvelukartta.hel.fi/unit/41960,,NaN,POINT (2781186.232 8450609.098)
70B3D57050001BBE,Rastilan uimaranta,60.207977,25.114849,https://palvelukartta.hel.fi/fi/unit/40157,,NaN,POINT (2795772.202 8446187.952)
70B3D57050004D86,Pihlajasaari,60.140588,24.915700,https://palvelukartta.hel.fi/fi/unit/45606,,NaN,POINT (2773603.059 8431104.992)
70B3D57050004FB9,Hietaniemi (Ourit),60.171500,24.898300,https://palvelukartta.hel.fi/fi/unit/41717,http://www.tuk.fi,NaN,POINT (2771666.078 8438019.854)
70B3D57050004C07,Sompasauna,60.175742,24.975318,https://palvelukartta.hel.fi/fi/unit/54929,https://www.sompasauna.fi,NaN,POINT (2780239.682 8438969.276)
70B3D57050004DF8,Vasikkasaari,60.152320,25.015860,https://palvelukartta.hel.fi/fi/unit/50903,https://www.vasikkasaari.org,NaN,POINT (2784752.797 8433728.617)
70B3D57050004FE1,Herttoniemi (Tuorinniemen uimalaituri),60.185540,25.040120,https://palvelukartta.hel.fi/fi/unit/41791,,NaN,POINT (2787453.408 8441162.681)
70B3D57050004FE6,Vartiosaari (Reposalmen laituri),60.180100,25.068600,https://palvelukartta.hel.fi/fi/unit/57156,https://www.vartiosaari.fi,NaN,POINT (2790623.787 8439944.788)
70B3D57050004E0E,Marjaniemen uimaranta,60.198449,25.076416,https://palvelukartta.hel.fi/fi/unit/40386,,NaN,POINT (2791493.860 8444053.522)
70B3D5705000504F,Hanikan uimaranta (Espoo),60.127797,24.691871,https://palvelukartta.hel.fi/fi/unit/39583,,NaN,POINT (2748686.506 8428245.610)


In [4]:
# Dropping the Uunisaari data since there is no data
geo = geo.drop(index="003C62A8")


In [5]:
geo

,name,lat,lon,servicemap_url,site_url,fieldmap,geometry
70B3D57050001AB9,Pikkukosken uimaranta,60.227704,24.983821,https://palvelukartta.hel.fi/unit/41960,,NaN,POINT (2781186.232 8450609.098)
70B3D57050001BBE,Rastilan uimaranta,60.207977,25.114849,https://palvelukartta.hel.fi/fi/unit/40157,,NaN,POINT (2795772.202 8446187.952)
70B3D57050004D86,Pihlajasaari,60.140588,24.915700,https://palvelukartta.hel.fi/fi/unit/45606,,NaN,POINT (2773603.059 8431104.992)
70B3D57050004FB9,Hietaniemi (Ourit),60.171500,24.898300,https://palvelukartta.hel.fi/fi/unit/41717,http://www.tuk.fi,NaN,POINT (2771666.078 8438019.854)
70B3D57050004C07,Sompasauna,60.175742,24.975318,https://palvelukartta.hel.fi/fi/unit/54929,https://www.sompasauna.fi,NaN,POINT (2780239.682 8438969.276)
70B3D57050004DF8,Vasikkasaari,60.152320,25.015860,https://palvelukartta.hel.fi/fi/unit/50903,https://www.vasikkasaari.org,NaN,POINT (2784752.797 8433728.617)
70B3D57050004FE1,Herttoniemi (Tuorinniemen uimalaituri),60.185540,25.040120,https://palvelukartta.hel.fi/fi/unit/41791,,NaN,POINT (2787453.408 8441162.681)
70B3D57050004FE6,Vartiosaari (Reposalmen laituri),60.180100,25.068600,https://palvelukartta.hel.fi/fi/unit/57156,https://www.vartiosaari.fi,NaN,POINT (2790623.787 8439944.788)
70B3D57050004E0E,Marjaniemen uimaranta,60.198449,25.076416,https://palvelukartta.hel.fi/fi/unit/40386,,NaN,POINT (2791493.860 8444053.522)
70B3D5705000504F,Hanikan uimaranta (Espoo),60.127797,24.691871,https://palvelukartta.hel.fi/fi/unit/39583,,NaN,POINT (2748686.506 8428245.610)


In [74]:
# Reading the temperature data
temps = pd.read_csv("https://iot.fvh.fi/opendata/uiras/uiras-all-data.csv",
                   parse_dates=['readable_time'],
                    usecols=["readable_time", "temp_out1", "dev-id"]
                   )


In [75]:
temps

,readable_time,temp_out1,dev-id
0,2020-01-31 22:08:19.096000+00:00,1.62,70B3D57050001AB9
1,2020-01-31 22:18:20.076000+00:00,1.62,70B3D57050001AB9
2,2020-01-31 22:28:22.252000+00:00,1.56,70B3D57050001AB9
3,2020-01-31 22:38:22.101000+00:00,1.62,70B3D57050001AB9
4,2020-01-31 22:48:23.015000+00:00,1.56,70B3D57050001AB9
...,...,...,...
139260,2020-11-30 15:50:22.012000+00:00,2.50,70B3D57050001AB9
139261,2020-11-30 15:50:47.329000+00:00,5.87,70B3D57050005037
139262,2020-11-30 15:52:30.139000+00:00,4.62,70B3D57050004FE1
139263,2020-11-30 15:55:20.107000+00:00,4.62,70B3D57050001BBE


In [76]:
# Reversing the dataframe
temps = temps[::-1]


In [77]:
temps.head(20)

,readable_time,temp_out1,dev-id
139264,2020-11-30 15:55:24.086000+00:00,4.06,70B3D57050004C07
139263,2020-11-30 15:55:20.107000+00:00,4.62,70B3D57050001BBE
139262,2020-11-30 15:52:30.139000+00:00,4.62,70B3D57050004FE1
139261,2020-11-30 15:50:47.329000+00:00,5.87,70B3D57050005037
139260,2020-11-30 15:50:22.012000+00:00,2.50,70B3D57050001AB9
139259,2020-11-30 15:45:22.405000+00:00,5.68,70B3D57050004FB9
139258,2020-11-30 15:44:29.971000+00:00,5.75,70B3D57050004D86
139257,2020-11-30 15:40:22.248000+00:00,2.50,70B3D57050001AB9
139256,2020-11-30 15:38:46.548000+00:00,3.93,70B3D57050004E0E
139255,2020-11-30 15:35:53.440000+00:00,3.06,70B3D57050001AF1


In [81]:
# Creating an empty list and dataframe
latest_temps = pd.DataFrame()
devices = []

# For-looping the temps dataframe
for index, row in temps.iterrows():
    
    # Checking if device-id has been already stored in the list
    if row["dev-id"] not in devices:

        # Appending rows to the new dataframe
        latest_temps = latest_temps.append(row)
        
        # Adding the device-id to the list
        devices.append(row["dev-id"])
    
    # Since I know there are only 14 devices,
    # no reason to iterate over the whole dataset
    if len(devices) == 14:
        break


In [82]:
devices

['70B3D57050004C07',
 '70B3D57050001BBE',
 '70B3D57050004FE1',
 '70B3D57050005037',
 '70B3D57050001AB9',
 '70B3D57050004FB9',
 '70B3D57050004D86',
 '70B3D57050004E0E',
 '70B3D57050001AF1',
 '70B3D57050004DF8',
 '70B3D57050001BA6',
 '70B3D57050004FE6',
 '70B3D5705000516A',
 '70B3D5705000504F']

In [85]:
import datetime as dt
# Creating a string from the timestamp object
latest_temps["time"] = latest_temps.readable_time.dt.strftime('%Y-%m-%d %H:%M')


In [92]:
latest_temps

,dev-id,temp_out1,time
139264,70B3D57050004C07,4.06,2020-11-30 15:55
139263,70B3D57050001BBE,4.62,2020-11-30 15:55
139262,70B3D57050004FE1,4.62,2020-11-30 15:52
139261,70B3D57050005037,5.87,2020-11-30 15:50
139260,70B3D57050001AB9,2.50,2020-11-30 15:50
139259,70B3D57050004FB9,5.68,2020-11-30 15:45
139258,70B3D57050004D86,5.75,2020-11-30 15:44
139256,70B3D57050004E0E,3.93,2020-11-30 15:38
139255,70B3D57050001AF1,3.06,2020-11-30 15:35
139254,70B3D57050004DF8,6.00,2020-11-30 15:34


In [94]:
# Removing the unnecessary columns
latest_temps = latest_temps[["dev-id", "temp_out1", "time"]]

geo = geo[["name", "geometry"]]

In [95]:
geo

,name,geometry
70B3D57050001AB9,Pikkukosken uimaranta,POINT (2781186.232 8450609.098)
70B3D57050001BBE,Rastilan uimaranta,POINT (2795772.202 8446187.952)
70B3D57050004D86,Pihlajasaari,POINT (2773603.059 8431104.992)
70B3D57050004FB9,Hietaniemi (Ourit),POINT (2771666.078 8438019.854)
70B3D57050004C07,Sompasauna,POINT (2780239.682 8438969.276)
70B3D57050004DF8,Vasikkasaari,POINT (2784752.797 8433728.617)
70B3D57050004FE1,Herttoniemi (Tuorinniemen uimalaituri),POINT (2787453.408 8441162.681)
70B3D57050004FE6,Vartiosaari (Reposalmen laituri),POINT (2790623.787 8439944.788)
70B3D57050004E0E,Marjaniemen uimaranta,POINT (2791493.860 8444053.522)
70B3D5705000504F,Hanikan uimaranta (Espoo),POINT (2748686.506 8428245.610)


In [97]:
# Merging the datasets
join = geo.merge(latest_temps, left_index=True, right_on="dev-id")


In [98]:
join

,name,geometry,dev-id,temp_out1,time
139260,Pikkukosken uimaranta,POINT (2781186.232 8450609.098),70B3D57050001AB9,2.50,2020-11-30 15:50
139263,Rastilan uimaranta,POINT (2795772.202 8446187.952),70B3D57050001BBE,4.62,2020-11-30 15:55
139258,Pihlajasaari,POINT (2773603.059 8431104.992),70B3D57050004D86,5.75,2020-11-30 15:44
139259,Hietaniemi (Ourit),POINT (2771666.078 8438019.854),70B3D57050004FB9,5.68,2020-11-30 15:45
139264,Sompasauna,POINT (2780239.682 8438969.276),70B3D57050004C07,4.06,2020-11-30 15:55
139254,Vasikkasaari,POINT (2784752.797 8433728.617),70B3D57050004DF8,6.00,2020-11-30 15:34
139262,Herttoniemi (Tuorinniemen uimalaituri),POINT (2787453.408 8441162.681),70B3D57050004FE1,4.62,2020-11-30 15:52
139251,Vartiosaari (Reposalmen laituri),POINT (2790623.787 8439944.788),70B3D57050004FE6,4.37,2020-11-30 15:26
139256,Marjaniemen uimaranta,POINT (2791493.860 8444053.522),70B3D57050004E0E,3.93,2020-11-30 15:38
139242,Hanikan uimaranta (Espoo),POINT (2748686.506 8428245.610),70B3D5705000504F,4.56,2020-11-30 15:07


In [ ]:
########
#import contextily as ctx
#fig, ax = plt.subplots()

#geo.plot(ax=ax, legend=True)

# Add basemap
#ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom=15)

In [144]:

import folium

my_tooltip = folium.features.GeoJsonTooltip(
    fields=["name"],
    aliases=["Place:"],
    labels=True,
    sticky=False
)


my_popup = folium.features.GeoJsonPopup(
    fields=["name", "time", "temp_out1"],
    aliases=["Place:", "Measured at:", "Temperature (°C):"],
    localize=True,
    parse_html=True
)

point_gjson = folium.features.GeoJson(
    join,
    name="Beaches",
    popup=my_popup,
    tooltip=my_tooltip
)

m = folium.Map(
    location=[60.25, 24.95],
    zoom_start=11,
    control_scale=True
)

point_gjson.add_to(m)

#folium.GeoJson(join,
#    tooltip=my_tooltip,
#    popup=my_popup).add_to(m)

#markers = folium.Marker(point_gjson)

folium.LayerControl().add_to(m)

#markers = folium.Marker(popup="Uimaranta", tooltip="kalupiste")

#markers.add_to(m)
    
m   
    

In [99]:
#import folium

#point_gjson = folium.features.GeoJson(join, name="points")

#m = folium.Map(location=[60.25, 24.8], zoom_start=10, control_scale=True)

#point_gjson.add_to(m)

#


In [161]:
geo

,name,lat,lon,servicemap_url,site_url,fieldmap,geometry
70B3D57050001AB9,Pikkukosken uimaranta,60.227704,24.983821,https://palvelukartta.hel.fi/unit/41960,,NaN,POINT (2781186.232 8450609.098)
70B3D57050001BBE,Rastilan uimaranta,60.207977,25.114849,https://palvelukartta.hel.fi/fi/unit/40157,,NaN,POINT (2795772.202 8446187.952)
70B3D57050004D86,Pihlajasaari,60.140588,24.915700,https://palvelukartta.hel.fi/fi/unit/45606,,NaN,POINT (2773603.059 8431104.992)
70B3D57050004FB9,Hietaniemi (Ourit),60.171500,24.898300,https://palvelukartta.hel.fi/fi/unit/41717,http://www.tuk.fi,NaN,POINT (2771666.078 8438019.854)
70B3D57050004C07,Sompasauna,60.175742,24.975318,https://palvelukartta.hel.fi/fi/unit/54929,https://www.sompasauna.fi,NaN,POINT (2780239.682 8438969.276)
70B3D57050004DF8,Vasikkasaari,60.152320,25.015860,https://palvelukartta.hel.fi/fi/unit/50903,https://www.vasikkasaari.org,NaN,POINT (2784752.797 8433728.617)
70B3D57050004FE1,Herttoniemi (Tuorinniemen uimalaituri),60.185540,25.040120,https://palvelukartta.hel.fi/fi/unit/41791,,NaN,POINT (2787453.408 8441162.681)
70B3D57050004FE6,Vartiosaari (Reposalmen laituri),60.180100,25.068600,https://palvelukartta.hel.fi/fi/unit/57156,https://www.vartiosaari.fi,NaN,POINT (2790623.787 8439944.788)
70B3D57050004E0E,Marjaniemen uimaranta,60.198449,25.076416,https://palvelukartta.hel.fi/fi/unit/40386,,NaN,POINT (2791493.860 8444053.522)
70B3D5705000504F,Hanikan uimaranta (Espoo),60.127797,24.691871,https://palvelukartta.hel.fi/fi/unit/39583,,NaN,POINT (2748686.506 8428245.610)


In [164]:
######
#url = 'https://a.basemaps.cartocdn.com/light_all/10/24.6/60.8.png'
#import os
#import urllib

# Control figure size in here
#fig, ax = plt.subplots(figsize=(12,8))

# The formatting should follow: 'https://{s}.basemaps.cartocdn.com/{style}/{z}/{x}/{y}{scale}.png'
# Specify the style to use
#style = "rastertiles/voyager"
#cartodb_url = 'https://a.basemaps.cartocdn.com/%s/{z}/{x}/{y}.png' % style

# Plot the data from subset
#geo.plot(ax=ax, legend=True)

#credits = "testi"

# Add basemap with `OSM_A` style using zoom level of 14 
#ctx.add_basemap(ax, zoom=10, attribution=credits, source=cartodb_url)

# Crop the figure
#ax.set_xlim(2770000, 2785000)
#ax.set_ylim(8435000, 8442500)

